<center><h1 style="color: #555555;">Experience Script Documentation</h1></center>

<div align="center">
    <table style="width: 80%; border-collapse: collapse;">
        <tr>
            <th style="background-color: #6AAFE6; color: #ffffff; padding: 10px;">Context</th>
            <th style="background-color: #6AAFE6; color: #ffffff; padding: 10px;">Approach</th>
            <th style="background-color: #6AAFE6; color: #ffffff; padding: 10px;">Value Created</th>
        </tr>
        <tr>
            <td style="background-color: #E8F4FC; padding: 10px;">Part of a currency conversion operation requiring monthly server-side updates with closing FX rates.</td>
            <td style="background-color: #E8F4FC; padding: 10px;">Applied configparser for secure API key retrival and encapsulated rate extraction using ForeignExchange class from Alpha Vantage.</td>
            <td style="background-color: #E8F4FC; padding: 10px;">Automated server-side FX rate updates, reducing processing time by 80% and eliminating human error risks.</td>
        </tr>
    </table>
</div>
</html>

In [1]:
from alpha_vantage.foreignexchange import ForeignExchange
import configparser
import pandas as pd
from datetime import datetime

# Initialize the configuration parser
config = configparser.ConfigParser()

# Load the configuration file
config.read('config.ini')

# Accessing The API key from Alpha Vantage from the configuration values
api_key = config['API']['API_KEY']
cc = ForeignExchange(key=api_key)

currency_pairs = {
    'CAD': 'USD',
    'EUR': 'USD',
    'GTQ': 'USD',
    'MXN': 'USD'
}

months_to_add = ['202404', '202405']  # Modify to desired months

new_fx_data = pd.DataFrame()

for base_currency, quote_currency in currency_pairs.items():
    try:
        data, _ = cc.get_currency_exchange_daily(from_symbol=base_currency, to_symbol=quote_currency, outputsize='full')
        
        # Convert the received data directly into a DataFrame
        df = pd.DataFrame.from_dict(data, orient='index', dtype=float).sort_index()
        df.index = pd.to_datetime(df.index)  # Ensure the index is datetime for easier filtering
        
        # Filter and process the data for each specified month
        for month in months_to_add:
            # Determine the start and end of the month for filtering
            start_date = datetime.strptime(f"{month}01", '%Y%m%d')
            end_date = start_date + pd.offsets.MonthEnd(1)
            
            # Filter the data for the month
            month_data = df.loc[(df.index >= start_date) & (df.index <= end_date)]
            
            if not month_data.empty:
                # Get the last available rate for the month
                closing_rate = month_data.iloc[-1]['4. close']
                
                new_fx_data = new_fx_data.append({
                    'CURRENCY': base_currency,
                    'PERIOD': month + '01',
                    'FX': closing_rate
                }, ignore_index=True)
    
    except Exception as e:
        print(f"An error occurred while processing {base_currency}/{quote_currency}: {e}")

# Check if new_fx_data is not empty and contains the 'FX' column before conversion
if not new_fx_data.empty and 'FX' in new_fx_data.columns:
    new_fx_data['FX'] = new_fx_data['FX'].astype(float)

print(new_fx_data)

D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_7976\2611687880.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fx_data = new_fx_data.append({
D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_7976\2611687880.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fx_data = new_fx_data.append({
D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_7976\2611687880.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fx_data = new_fx_data.append({
D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_7976\2611687880.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fx_data = new_fx_data.append({
D:\U

  CURRENCY    PERIOD       FX
0      CAD  20240401  0.72582
1      CAD  20240501  0.73330
2      EUR  20240401  1.06665
3      EUR  20240501  1.08354
4      GTQ  20240401  0.12860
5      GTQ  20240501  0.12867
6      MXN  20240401  0.05824
7      MXN  20240501  0.05875


D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_7976\2611687880.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fx_data = new_fx_data.append({
D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_7976\2611687880.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_fx_data = new_fx_data.append({


In [2]:
# Read the original table
dtype = {
    'CURRENCY':'str',
    'PERIOD':'str',
    'FX': float,
    }

# Define the input and output paths directly
input_path = r'D:\\Users\\diana.fernandez\\Downloads\\TBC\\LOOKUP TABLES\\FX\\'
output_path = r'D:\\Users\\diana.fernandez\\Downloads\\TBC\\LOOKUP TABLES\\FX\\'

df = pd.read_csv(input_path + 'fx_rate_server_side_table_B4_6.11.24.csv', encoding='UTF-8-SIG', dtype=dtype)

df_shape = df.shape
print(f"Server-side before shape {df_shape}")

print(df)

Server-side before shape (940, 3)
    CURRENCY    PERIOD       FX
0        CAD  20190101  0.76200
1        EUR  20190101  1.14440
2        GTQ  20190101  0.12850
3        MXN  20190101  0.05230
4        CAD  20190201  0.75930
..       ...       ...      ...
935      MXN  20240201  0.05861
936      CAD  20240301  0.73810
937      EUR  20240301  1.07900
938      GTQ  20240301  0.12827
939      MXN  20240301  0.06027

[940 rows x 3 columns]


In [3]:
# If the DataFrames have extra columns, drop them
columns_to_keep = ['CURRENCY', 'PERIOD', 'FX']
new_fx_data = new_fx_data[columns_to_keep]

# Append the new FX rates to the original DataFrame, ensuring it's at the bottom
updated_df = pd.concat([df, new_fx_data], ignore_index=True)

# Confirm the DataFrame has only the desired columns
updated_df = updated_df[columns_to_keep]

# Get the updated DataFrame shape for verification
updated_df_shape = updated_df.shape

# Export the updated DataFrame to a new CSV file with a timestamp
timestamp = datetime.now().strftime('%m.%d.%y')  # Format: mm.dd.yy
filename = output_path + f'fx_rate_server_side_table_AFT_{timestamp}.csv'
updated_df.to_csv(filename, index=False, encoding='UTF-8-SIG')

# Print the shape of the updated DataFrame and filename
print(f"Server-side after shape: {updated_df_shape}")
print(f"Data exported to: {filename}")


Server-side after shape: (948, 3)
Data exported to: D:\\Users\\diana.fernandez\\Downloads\\TBC\\LOOKUP TABLES\\FX\\fx_rate_server_side_table_AFT_06.11.24.csv
